### CS4102 - Geometric Foundations of Data Analysis I
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

# Week 11: Voronoi Cells.

* How to construct (plot) the Voronoi diagram of a given set of points in $\mathbb{R}^2$.

![voronoi](https://upload.wikimedia.org/wikipedia/commons/thumb/5/54/Euclidean_Voronoi_diagram.svg/573px-Euclidean_Voronoi_diagram.svg.png)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

###  Bisectors

* Here are some points to play with:

In [ ]:
pp = np.random.rand(4, 2)
pp

* `plot` expects $x$-coordinates, followed by $y$-coordinates.
* Thats column $0$ of `pp`, followed by column $1$ ...

In [ ]:
plt.plot(pp[:,0], pp[:,1], 'ro')

* ... or `pp` transposed and unpacked with the list unpacking operator `*`

In [ ]:
[*pp.T]

In [ ]:
pp.T

In [ ]:
plt.plot(*pp.T, 'ro')

* Let's try and construct (i.e. plot) the bisector of the first two points, $A$ and $B$.

In [ ]:
A = pp[0]
B = pp[1]
A, B

* The bisector of $A$ and $B$ is the line $\ell$ through the midpoint $M = \frac12(A + B)$ with slope $m = -1/m'$, where $m' = \frac{a_1 - b_1}{a_0 - b_0}$ is the slope of the line through $A = (a_0, a_1)$ and $B = (b_0, b_1)$.
* It follows that the equation of $\ell$ is $y = mx + c$, where $c = m_1 - m\, m_0$ if $M = (m_0, m_1)$. 

In [ ]:
M = 1/2 * (A + B)
AmB = A - B
m = - AmB[0]/AmB[1]
m

In [ ]:
c = M[1] - m * M[0]
c

* `plot` will produce a line segment if it is given two points as argument.
* Here, as $0 \leq x \leq 1$, we can plot $\ell$ as the line connecting the points $(0, y_0)$ and $(1, y_1)$ on $\ell$.
* And since the $x$-values default to $0$ and $1$ anyway, it suffices to supply $y_0 = c$ and $y_1 = m + c$.
* (That's quick and dirty rather than sustainable - but for now we don't know yet whether this needs to be sustained ...)

In [ ]:
ys = np.array([0, m]) + c
ys

In [ ]:
plt.plot(*pp[:2].T, 'ro')
plt.plot(ys)

* The line doesn't quite look perpendicular. That's because there are different scales on $x$ and $y$-axes.

In [ ]:
def bisector(A, B):
    M = 1/2 * (A + B)
    AmB = (A - B)
    m = - AmB[0]/AmB[1]
    c = M[1] - m * M[0]
    return np.array([0, m]) + c

In [ ]:
bisector(A, B)

In [ ]:
plt.xlim(0,1)
plt.ylim(0,1)
plt.plot(*np.array([A, B]).T, 'ro')
plt.plot(bisector(A, B))

* Now, let's plot all the points, and all the bisectors!  How many bisectors are there?

In [ ]:
plt.xlim(0,1)
plt.ylim(0,1)
plt.plot(*pp.T, 'ro')
for i, A in enumerate(pp):
    for j in range(i):
        B = pp[j]
        plt.plot(bisector(A, B))

* Very often, three lines seem to intersect in a single point.  That cannot be a coincidence ...

### Circumcircle

* A formula for the centre and radius of the circumcircle of (a triangle specified by) three points $A$, $B$, and $C$ can be found on [wikipedia](https://en.wikipedia.org/wiki/Circumscribed_circle#Circumcircle_equations).
* Let's look at the first three points in our list `pp`.

In [ ]:
A, B, C = pp[:3]
A, B, C

In [ ]:
plt.plot(*(pp[:3].T), 'ro')

* Shifting all points by $-A$ (so that $A$ becomes the origin $O = A - A$) simplifies the calculations

In [ ]:
B1 = B - A
C1 = C - A
B1, C1

* We need the squared lengths $b' = \|B'\|^2$ and $c' = \|C'\|^2$ of the shifted vectors

In [ ]:
b1 = B1 @ B1
c1 = C1 @ C1
b1, c1

* A rotation $R$ about $90^{\circ}$ also plays a role.

In [ ]:
rot = np.array([[0, -1], [1, 0]])
rot

In [ ]:
B1 @ rot

* Now, the circumcentre of $O$, $A'$ and $B'$ is
\\[
  U' = \frac1{d'} (b' C' - c' B').R
\\]
where $d' = 2 B'.(C'.R)^T$.
* And the radius is simply $r = \| U' \|$

In [ ]:
d1 = 2 * np.dot(B1, C1 @ rot)
d1

In [ ]:
U1 = (b1 * C1 - c1 * B1) @ rot / d1
U1

In [ ]:
r = np.sqrt(U1 @ U1)
r

* The circumcentre of $A$, $B$ and $C$ lies at $U' + A$.

In [ ]:
U1 + A

In [ ]:
circle = plt.Circle(U1 + A, r, color= 'm', fill=False, alpha=0.5)
plt.plot(*(pp[:3].T), 'ro')
plt.gca().add_patch(circle)

* Spot on!

In [ ]:
def circumcircle(A, B, C):
    B1, C1 = [B, C] - A
    b1 = B1 @ B1
    c1 = C1 @ C1
    rot = np.array([[0, -1], [1, 0]])  
    d1 = 2 * np.dot(B1, C1 @ rot)
    
    U1 = (b1 * C1 - c1 * B1) @ rot / d1
    r = np.sqrt(U1 @ U1)
    return U1 + A, r

In [ ]:
circumcircle(A, B, C)

* Let's plot all the points and all their circumcentres. How many circumcentres are there?

In [ ]:
plt.plot(*pp.T, 'ro')
circles = []
for i, A in enumerate(pp):
    for j in range(i):
        B = pp[j]
        for k in range(j):
            C = pp[k]
            centre, radius = circumcircle(A, B, C)
            circles.append({ "c" : centre, "r" : radius })
            circle = plt.Circle(centre, radius, alpha=0.2)
            plt.plot(centre[0], centre[1], 'kx')
            plt.gca().add_patch(circle)

In [ ]:
circles

##  Exercises

1. Inspect the list of circumcircles in this and other examples of random points.  Which attributes of (some of) the circles look like they can contribute to the construction of Voronoi cells? Can you find a criterion that distinguishes useful circles from the others?

1. Inspect the bisectors in this and other examples of random points.  Which attributes of (some of) the bisecting lines look like they can contribute to the construction of Voronoi cells? Can you find a criterion that distinguishes useful lines from the others?

1. In the current context, what is the meaning of the line segement connecting two centers of circumcircles?  What difference does the number of common data points on the two circumcircles make?